In [2]:
import steamreviews
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import os

In [3]:
request_params = dict()
request_params['language'] = 'english'

#Import the reviews from Steam
app_ids = [1145360, 462770, 237930, 107100]
steamreviews.download_reviews_for_app_id_batch(app_ids)

Loading idprocessed_on_20230703.txt
Skipping previously found appID = 1145360
Skipping previously found appID = 462770
Skipping previously found appID = 237930
Skipping previously found appID = 107100
Game records written: 0


True

In [22]:
# List all JSON files in a directory
json_files = [pos_json for pos_json in os.listdir('D:/Git/phase_4/data') if pos_json.endswith('.json')]

# Create an empty DataFrame to store all reviews
df = pd.DataFrame()

# Loop through all JSON files and concatenate them into the DataFrame
for file in json_files:
    temp_df = pd.read_json('D:/Git/phase_4/data/' + file)
    df = pd.concat([df, temp_df], ignore_index=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301971 entries, 0 to 301970
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   reviews        301939 non-null  object
 1   query_summary  24 non-null      object
 2   cursors        8 non-null       object
dtypes: object(3)
memory usage: 6.9+ MB


In [24]:
# Only keep rows where 'reviews' is a dictionary
df = df[df['reviews'].apply(lambda x: isinstance(x, dict))]

# Normalize the 'reviews' column
expanded_reviews_df = pd.json_normalize(df['reviews'])

# Use the pd.json_normalize() function to expand the dictionaries in each row into their own dataframe
expanded_reviews_df = pd.json_normalize(df['reviews'])

# Concatenate it with your original dataframe if necessary
df = pd.concat([df, expanded_reviews_df], axis=1)

# Drop the original 'reviews' column as it is no longer needed
df = df.drop(columns=['reviews'])

# Save your dataframe to a CSV
df.to_csv('SuperGiant_reviews.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301963 entries, 0 to 293645
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   query_summary                   0 non-null       object 
 1   cursors                         0 non-null       object 
 2   recommendationid                301939 non-null  object 
 3   language                        301939 non-null  object 
 4   review                          301939 non-null  object 
 5   timestamp_created               301939 non-null  float64
 6   timestamp_updated               301939 non-null  float64
 7   voted_up                        301939 non-null  object 
 8   votes_up                        301939 non-null  float64
 9   votes_funny                     301939 non-null  float64
 10  weighted_vote_score             301939 non-null  object 
 11  comment_count                   301939 non-null  float64
 12  steam_purchase  

In [19]:
df.head()

,query_summary,cursors,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,...,steam_china_location,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,timestamp_dev_responded,developer_response
0,NaN,NaN,140062592,english,"After playing and absolutely loving Hades, I d...",1.686771e+09,1.686771e+09,True,0.0,0.0,...,,76561198182147498,0.0,14.0,318.0,318.0,318.0,1.686621e+09,NaN,NaN
1,NaN,NaN,140046743,english,"The game is great, combat is fun with lots of ...",1.686749e+09,1.686749e+09,True,0.0,0.0,...,,76561198405922322,180.0,6.0,392.0,392.0,392.0,1.686748e+09,NaN,NaN
9,NaN,NaN,139914326,english,"Finished this game off-steam, but have to come...",1.686520e+09,1.686520e+09,True,0.0,0.0,...,,76561197992689577,0.0,124.0,233.0,0.0,233.0,1.546734e+09,NaN,NaN
10,NaN,NaN,139882729,english,"[h1]Comentario Analítico: ""Bastion""[/h1]\n\nEn...",1.686478e+09,1.686478e+09,True,3.0,0.0,...,,76561198113848693,0.0,271.0,672.0,0.0,672.0,1.560983e+09,NaN,NaN
11,NaN,NaN,139877123,english,Supergiants first game and you can tell it's m...,1.686469e+09,1.686469e+09,True,0.0,0.0,...,,76561198281072985,395.0,24.0,279.0,0.0,279.0,1.641619e+09,NaN,NaN


In [30]:
#Drop Nulls
df = df.dropna(subset=['review'])

#Keep only English reviews
df = df[df['language'] == 'english']

# Drop Unnecessary Columns
df = df.drop(df.columns[[0, 1, 2, 5, 6, 15, 16, 17]], axis=1)

# Create a mask where each review has more than one word and at least one alphabetic character
mask = df['review'].apply(lambda x: len(re.findall(r'\b\w+\b', str(x))) > 5 and bool(re.search('[a-zA-Z]', str(x))))

# Apply the mask to the DataFrame to filter out reviews
df = df[mask]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115959 entries, 0 to 293645
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   language                        115959 non-null  object 
 1   review                          115959 non-null  object 
 2   voted_up                        115959 non-null  object 
 3   votes_up                        115959 non-null  float64
 4   votes_funny                     115959 non-null  float64
 5   weighted_vote_score             115959 non-null  object 
 6   comment_count                   115959 non-null  float64
 7   steam_purchase                  115959 non-null  object 
 8   received_for_free               115959 non-null  object 
 9   written_during_early_access     115959 non-null  object 
 10  author.num_games_owned          115959 non-null  float64
 11  author.num_reviews              115959 non-null  float64
 12  author.playtime_

In [33]:
df.head()

,language,review,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,timestamp_dev_responded,developer_response
0,english,"After playing and absolutely loving Hades, I d...",True,0.0,0.0,0,0.0,True,False,False,0.0,14.0,318.0,318.0,318.0,1.686621e+09,NaN,NaN
1,english,"The game is great, combat is fun with lots of ...",True,0.0,0.0,0,0.0,True,False,False,180.0,6.0,392.0,392.0,392.0,1.686748e+09,NaN,NaN
9,english,"Finished this game off-steam, but have to come...",True,0.0,0.0,0,0.0,True,False,False,0.0,124.0,233.0,0.0,233.0,1.546734e+09,NaN,NaN
10,english,"[h1]Comentario Analítico: ""Bastion""[/h1]\n\nEn...",True,3.0,0.0,0.559255659580230713,0.0,False,False,False,0.0,271.0,672.0,0.0,672.0,1.560983e+09,NaN,NaN
11,english,Supergiants first game and you can tell it's m...,True,0.0,0.0,0,0.0,True,False,False,395.0,24.0,279.0,0.0,279.0,1.641619e+09,NaN,NaN


In [11]:
# Now, drop the columns
df_process = pd.DataFrame(df['review'])

In [12]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149410 entries, 0 to 293645
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  149410 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [13]:
df_process['review'] = df_process['review'].astype(str)

In [14]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149410 entries, 0 to 293645
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  149410 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [18]:
df_process.drop('review', axis=1, inplace=True)

In [23]:
print(df_process['processed_review'].dtypes)

object


In [22]:
# Convert all values in the 'processed_review' column to strings
df_process['processed_review'] = df_process['processed_review'].apply(str)

# Check the data types in the 'processed_review' column
print(df_process['processed_review'].dtypes)

object


In [15]:
df_process.to_csv('reviews_processed.csv', index=False)

Lemmatize?

In [ ]:
# Instantiate a PorterStemmer
ps = PorterStemmer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = re.sub('[^a-z]', ' ', text)
    
    # Split the text into words and remove stopwords, then apply stemming
    text = ' '.join(ps.stem(word) for word in text.split() if word not in set(stopwords.words('english')))
    
    return text

# Apply the preprocessing to each review
df_process['processed_review'] = df_process['review'].apply(preprocess_text)

# Filter out any reviews that are now just 'nan'
df_process = df_process[df_process['processed_review'] != 'nan']

In [ ]:
vectorizer = TfidfVectorizer(max_features=2500) # Use the top 2500 words as features
X = vectorizer.fit_transform(df['review']).toarray()

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df['label']) # Assuming your labels column is named 'label'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print("Classification Report: \n", classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("Accuracy: \n", accuracy_score(y_test, y_pred))